In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
import os
from matplotlib import pyplot as plt, cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.viewer import ImageViewer
import skimage.io
import matplotlib.colors as colors
import numpy as np

import imgfileutils as imf

In [2]:
basefolder = r'/datadisk1/tuxedo/IPython_Notebooks/testdata'
#basefolder = r'/home/sebi06/testdata'

#filename_ometiff = os.path.join(basefolder, 'Filter_with_Particles_big.ome.tiff')
filename_ometiff = os.path.join(basefolder, 'S=2_T=5_CH=3_CH=2_A2.ome.tiff')
#filename_ometiff = os.path.join(basefolder, 'Osteosarcoma_01.ome.tiff')
#filename_czi = os.path.join(basefolder, 'Filter_with_Particles_big.czi')
#filename_czi = os.path.join(basefolder, '8Brains_DAPI_5X_stitched.czi')
#filename_czi = os.path.join(basefolder, 'S=2_T=5_Z=3_CH=2_A2.czi')

In [3]:
image_name = os.path.basename(filename_ometiff)

if filename_ometiff.lower().endswith('.ome.tiff') or filename_ometiff.lower().endswith('.ome.tif'):
    
    # Return value is an 5D Array of order (T, Z, C, X, Y)
    (array5d, omexml) = io.read_ometiff(filename_ometiff)
    
metadata = imf.get_metadata_ometiff(filename_ometiff, omexml, series=0)

In [4]:
array5d.shape

(5, 3, 2, 1216, 1216)

In [5]:
metadata

{'Directory': '/datadisk1/tuxedo/IPython_Notebooks/testdata',
 'Filename': 'S=2_T=5_CH=3_CH=2_A2.ome.tiff',
 'Name': 'S=2_T=5_Z=3_CH=2_A2.czi #1',
 'AcqDate': '2014-08-27T10:51:22.039',
 'TotalSeries': 1,
 'SizeX': 1216,
 'SizeY': 1216,
 'SizeZ': 3,
 'SizeC': 2,
 'SizeT': 5,
 'DimOrder BF': 'XYCZT',
 'NA': '0.025',
 'ObjMag': '1.0',
 'ObjID': 'Objective:1',
 'XScale': 1.0,
 'YScale': 1.0,
 'ZScale': 3.0,
 'XScaleUnit': 'µm',
 'YScaleUnit': 'µm',
 'ZScaleUnit': 'µm',
 'DetectorModel': None,
 'DetectorName': [],
 'DetectorID': 'Detector:Camera',
 'InstrumentID': 'Instrument:0',
 'Channels': ['DAPI', 'EGFP'],
 'ImageIDs': [0]}

In [6]:
import ipywidgets as widgets

t = widgets.IntSlider(description='T:',
                      min=1,
                      max=metadata['SizeT'],
                      step=1,
                      value=1,
                      continuous_update=False)

z = widgets.IntSlider(description='Z:',
                      min=1,
                      max=metadata['SizeZ'],
                      step=1,
                      value=1,
                      continuous_update=False)

c = widgets.IntSlider(description='C:',
                      min=1,
                      max=metadata['SizeC'],
                      step=1,
                      value=1)

r = widgets.IntRangeSlider(description='Display Range:',
                           min=array5d.min(),
                           max=array5d.max(),
                           step=1,
                           value=[array5d.min(), array5d.max()],
                           continuous_update=False)

ui = widgets.VBox([t, z, c, r])

In [7]:
def display_image(array5d, t=0, c=0, z=0, vmin=0, vmax=1000):
    image = array5d[t-1, z-1, c-1, :, :]
    # display the labelled image
    fig, ax = plt.subplots(figsize=(10, 10))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    im = ax.imshow(image, vmin=vmin, vmax=vmax, interpolation='nearest', cmap=cm.gray)
    fig.colorbar(im, cax=cax, orientation='vertical')
    print('Min-Max (Current Plane):', image.min(), '-', image.max())

def get_TZC(t, z, c, r):
    
    #minvalue = array5d[t-1, z-1, c-1, :, :].min()
    #maxvalue = array5d[t-1, z-1, c-1, :, :].max()
    #r.min = minvalue
    #r.max = maxvalue
    display_image(array5d, t=t, z=z, c=c, vmin=r[0], vmax=r[1])

# disable slider that are not needed
if metadata['SizeT'] == 1:
    t.disabled=True
if metadata['SizeZ'] == 1:
    z.disabled=True
if metadata['SizeC'] == 1:
    c.disabled=True

In [8]:
out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
display(ui, out)

Output()